In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
fnb = pd.read_csv('/content/drive/MyDrive/newnutcomp.csv')
fnb.head()

,food_id,Composition,Calories,Protein,Fat,Carbohydrate,Sugar
0,1001,"BUTTER,WITH SALT",717,0.85,81.11,0.06,0.06
1,1002,"BUTTER,WHIPPED,W/ SALT",718,0.49,78.30,2.87,0.06
2,1003,"BUTTER OIL,ANHYDROUS",876,0.28,99.48,0.00,0.00
3,1004,"CHEESE,BLUE",353,21.40,28.74,2.34,0.50
4,1005,"CHEESE,BRICK",371,23.24,29.68,2.79,0.51


In [3]:
allergen = pd.read_csv('/content/drive/MyDrive/newallergen.csv')
allergen.head()

,Class,Type,Group,Food,Allergy
0,Plant origin,Nut and seed,Oil seed,Almond,Nut Allergy
1,Plant origin,Fruit,Pome fruit,Apple,Oral Allergy Syndrome
2,Plant origin,Fruit,Stone fruit,Apricot,Stone Fruit Allergy
3,Plant origin,Vegetable,Composite vegetable,Artichoke,Insulin Allergy
4,Plant origin,Vegetable,Liliaceous vegetable,Asparagus,Allium Allergy


In [4]:
# Define the function to classify food based on nutritional values
def classify_food(row):
    if row['Sugar'] < 5 and row['Fat'] < 5 and row['Protein'] > 15 and 200 <= row['Calories'] <= 300:
        return 'A'
    elif 5 < row['Sugar'] <= 10 and 5 < row['Fat'] <= 10 and 10 < row['Protein'] <= 15 and 300 < row['Calories'] <= 400:
        return 'B'
    elif 10 < row['Sugar'] <= 20 and 10 < row['Fat'] <= 20 and row['Protein'] <= 10 and 400 < row['Calories'] <= 600:
        return 'C'
    else:
        return 'D'

# Add the Grade_food column to the fnb dataset
fnb['Grade_food'] = fnb.apply(classify_food, axis=1)
grade_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
fnb['Grade_food'] = fnb['Grade_food'].map(grade_mapping)

# Define features and target
X = fnb[['Sugar', 'Fat', 'Protein', 'Calories']]
y = fnb['Grade_food']

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost Classifier
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train the model
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Function to get recommendation messages
def get_message(classification, has_diabetes=False):
    if classification == 'A':
        message = """Grade: 'A' Healthy Food
Overall: Excellent for health. This food is nutrient-dense and can help maintain weight and heart health
Sugar: Low (less than 5g per serving)
Fat: Low (less than 5g per serving, with minimal saturated fat)
Protein: High (more than 15g per serving)
Calories: Moderate (200-300 calories per serving)"""
        if has_diabetes:
            message += "\nNote: This food is suitable for individuals with diabetes due to its low sugar content."
    elif classification == 'B':
        message = """Grade: 'B' Good Food
Overall: Good for health. This food can be part of a balanced diet, although portion size and frequency of consumption should be monitored
Sugar: Moderate (5-10g per serving)
Fat: Moderate (5-10g per serving, with controlled saturated fat)
Protein: Moderate (10-15g per serving)
Calories: Moderate (300-400 calories per serving)"""
        if has_diabetes:
            message += "\nNote: Individuals with diabetes should consume this food in moderation due to its moderate sugar content."
    elif classification == 'C':
        message = """Grade: 'C' Fair Food
Overall: Should be consumed with caution. This food can be an occasional choice, but it should not be a mainstay in daily diet
Sugar: High (10-20g per serving)
Fat: High (10-20g per serving, with some saturated fat)
Protein: Low (less than 10g per serving)
Calories: High (400-600 calories per serving)"""
        if has_diabetes:
            message += "\nWarning: This food contains high sugar levels and is not recommended for individuals with diabetes."
    else:
        message = """Grade : 'D' Unhealthy Food
Overall: Not recommended for regular consumption. This food can contribute to health issues such as obesity, diabetes, and heart disease
Sugar: Very high (more than 20g per serving)
Fat: Very high (more than 20g per serving, with high saturated fat)
Protein: Very low (less than 5g per serving)
Calories: Very high (more than 600 calories per serving)"""
        if has_diabetes:
            message += "\nWarning: This food is extremely high in sugar and is unsafe for individuals with diabetes."
    return message

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:05:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [5]:
# Initialize an empty dictionary to store the user inputs
user_inputs = {}

# Ask for food ID
user_inputs['food_id'] = int(input("Enter the food you are selecting: "))

# Ask for user details
user_inputs['name'] = input("Enter your name: ")
user_inputs['age'] = int(input("Enter your age: "))
user_inputs['weight'] = float(input("Enter your weight (in kg): "))
user_inputs['height'] = float(input("Enter your height (in cm): "))
user_inputs['diabetes'] = input("Do you have diabetes? (Yes/No): ")

# Ask for allergies
user_inputs['allergy'] = input("Enter any allergies you have (e.g., Lactose): ")

# Retrieve food data based on food_id
selected_food = fnb[fnb['food_id'] == user_inputs['food_id']].iloc[0]

# Classify the food
classification = classify_food(selected_food)
message = get_message(classification, user_inputs ['diabetes'].strip().lower() == 'yes')

Enter the food you are selecting: 1001
Enter your name: Ryan
Enter your age: 21
Enter your weight (in kg): 50
Enter your height (in cm): 160
Do you have diabetes? (Yes/No): yes
Enter any allergies you have (e.g., Lactose): lactose


In [6]:
# Final output
print(f"Product Details")
print(message)

# Retrieve the food composition based on food_id
food_info = fnb[fnb['food_id'] == user_inputs['food_id']]

if not food_info.empty:
    composition = food_info['Composition'].values[0]
    print(f"Contains: {composition}")

    # Check if the composition contains food from the allergen dataset
    allergen_matches = allergen[allergen['Food'].str.contains(composition.split(',')[0], case=False, na=False)]

# IF there are matches, check if the user's allergy is relevant
if not allergen_matches.empty:
    found_allergy = False
    for index, row in allergen_matches.iterrows():
        if user_inputs['allergy'].lower() in row['Allergy'].lower():
            print(f"\n\033[1mWarning:\033[0m {user_inputs['food_id']} may contain {row['Food']} which is related to {row['Allergy']}. Please avoid it.")
            found_allergy = True
            break

    if not found_allergy:
        print(f"\n{user_inputs['food_id']} appears SAFE based on your allergy.")
else:
    print(f"\n{user_inputs['food_id']} appears SAFE based on your allergy.")

Product Details
Grade : 'D' Unhealthy Food
Overall: Not recommended for regular consumption. This food can contribute to health issues such as obesity, diabetes, and heart disease
Sugar: Very high (more than 20g per serving)
Fat: Very high (more than 20g per serving, with high saturated fat)
Protein: Very low (less than 5g per serving)
Calories: Very high (more than 600 calories per serving)
Contains: BUTTER,WITH SALT



In [7]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Accuracy: {accuracy}")
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.9988623435722411
Precision: 0.7497144488863506
Recall: 0.7083333333333334
F1 score: 0.7271298701298702


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
#  Food filter predicted to have grade 'A'
grade_a_foods = fnb[fnb['Grade_food'] == 0]

# Result
print("Grade A Food:")
print(grade_a_foods[['Composition', 'Sugar', 'Fat', 'Protein', 'Calories']])

Grade A Food:
                                            Composition  Sugar   Fat  Protein  \
252                                  SPICES,BASIL,DRIED   1.71  4.07    22.98   
257                                       CHERVIL,DRIED   0.00  3.90    23.20   
266                                     DILL WEED,DRIED   0.00  4.36    19.96   
4499                          COD,ATLANTIC,DRIED&SALTED   0.00  2.37    62.82   
4655                        WHELK,UNSPEC,CKD,MOIST HEAT   0.00  0.80    47.68   
6540                                   WHEAT BRAN,CRUDE   0.41  4.25    15.55   
8215      SEAWEED,CANADIAN CULTIVATED EMI-TSUNOMATA,DRY   0.00  1.39    15.34   
8347                  WALRUS,MEAT,DRIED (ALASKA NATIVE)   0.00  2.60    57.00   
8391           CARIBOU,SHLDR MEAT,DRIED (ALASKA NATIVE)   0.00  3.67    59.38   
8404  SEAL,BEARDED (OOGRUK),MEAT,PART DRIED (ALASKA ...   0.00  2.31    62.38   

      Calories  
252        233  
257        237  
266        253  
4499       290  
4655     

In [9]:
#  Food filter predicted to have grade 'B'
grade_b_foods = fnb[fnb['Grade_food'] == 1]

# Result
print("Grade B Food:")
print(grade_b_foods[['Composition', 'Sugar', 'Fat', 'Protein', 'Calories']])

Grade B Food:
                                            Composition  Sugar   Fat  Protein  \
313                    SEASONING MIX,DRY,CHILI,ORIGINAL   8.67  7.30    10.82   
5621                        BREAD CRUMBS,DRY,GRATED,PLN   6.20  5.30    13.35   
5830  ROLLS, DINNER, PLN,COMM PREPARED (INC BROWN -N...   5.55  6.47    10.86   
5861                   BREAD CRUMBS,DRY,GRATED,SEASONED   5.74  5.48    14.13   
7825                     KASHI,TLC,PITA CRISPS,SEA SALT   5.40  9.20    10.50   
8249    MACARONI OR NOODLES W/ CHS,MICROWAVEABLE,UNPREP   8.20  6.56    11.48   

      Calories  
313        335  
5621       395  
5830       310  
5861       383  
7825       381  
8249       388  


In [10]:
#  Food filter predicted to have grade 'C'
grade_c_foods = fnb[fnb['Grade_food'] == 2]

# Result
print("Grade C Food:")
print(grade_c_foods[['Composition', 'Sugar', 'Fat', 'Protein', 'Calories']])

Grade C Food:
                                            Composition  Sugar    Fat  \
461                               BABYFOOD,CRACKERS,VEG  13.00  19.60   
1559                              SAUCE,CHS SAU MIX,DRY  10.26  18.33   
5569                    BISCUITS,PLN OR BTTRMLK,DRY MIX  11.65  15.40   
5689                                   COOKIES,MOLASSES  17.60  12.80   
5729                                      CRACKERS,MILK  10.32  13.77   
5738                             CRACKERS,WHEAT,REGULAR  15.48  16.40   
5998                             CRACKERS,WHEAT,RED FAT  15.01  13.37   
6010         CRACKERS,CRM,LA MODERNA RIKIS CRM CRACKERS  10.41  19.50   
6035  AUSTIN,CHEDDAR CHS ON CHS CRACKERS,SANDWICH-TY...  14.60  17.50   
6037  AUSTIN,PNUT BUTTER ON TOASTY CRACKERS,SANDWICH...  11.80  18.00   
6088  SNACKS,M&M MARS,COMBOS SNACKS CHEDDAR CHS PRETZEL  17.60  16.92   
6886                    FAST FOODS,MINIATURE CINN ROLLS  17.98  17.95   
7872             KEEBLER,CLUB,MINIS M

In [11]:
#  Food filter predicted to have grade 'D'
grade_d_foods = fnb[fnb['Grade_food'] == 3]

# Result
print("Grade D Food:")
print(grade_d_foods[['Composition', 'Sugar', 'Fat', 'Protein', 'Calories']])

Grade D Food:
                     Composition  Sugar    Fat  Protein  Calories
0               BUTTER,WITH SALT   0.06  81.11     0.85       717
1         BUTTER,WHIPPED,W/ SALT   0.06  78.30     0.49       718
2           BUTTER OIL,ANHYDROUS   0.00  99.48     0.28       876
3                    CHEESE,BLUE   0.50  28.74    21.40       353
4                   CHEESE,BRICK   0.51  29.68    23.24       371
...                          ...    ...    ...      ...       ...
8785             MACKEREL,SALTED   0.00  25.10    18.50       305
8786  SCALLOP,(BAY&SEA),CKD,STMD   0.00   0.84    20.54       111
8787                  SYRUP,CANE  73.20   0.00     0.00       269
8788                   SNAIL,RAW   0.00   1.40    16.10        90
8789            TURTLE,GREEN,RAW   0.00   0.50    19.80        89

[8754 rows x 5 columns]


In [12]:
grade_a_foods.to_csv("FoodResult_GradeA.csv", index=False)

In [13]:
grade_b_foods.to_csv("FoodResult_GradeB.csv", index=False)

In [14]:
grade_c_foods.to_csv("FoodResult_GradeC.csv", index=False)

In [15]:
grade_d_foods.to_csv("FoodResult_GradeD.csv", index=False)

In [16]:
# Save the model to a file
model.save_model('model3.xgb')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:06:01] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [17]:
from xgboost import XGBClassifier

# Create a new instance of the classifier
loaded_model = XGBClassifier()

# Load the model from the file
loaded_model.load_model('model3.xgb')

In [19]:
import tensorflow as tf

# Load your XGBoost model
xgb_model = model
xgb_model.load_model('model3.xgb')

# Create a TensorFlow model (this is just an example, adjust as needed)
inputs = tf.keras.Input(shape=(4,))
x = tf.keras.layers.Dense(10, activation='relu')(inputs)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
tf_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
tf_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow model on the same data (or use the XGBoost model's predictions)
# For demonstration, we will skip training and directly convert the model.

# Convert the TensorFlow model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('model3.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp6qv9ecrt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  134445338319984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134445338324912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134445338324384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134445338169184: TensorSpec(shape=(), dtype=tf.resource, name=None)
